In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from Class_utils.parameters import file_paths,job_graph_par
from Class_utils.JobGraph import JobGraph
import ast
from Class_utils.FitnessFunctions import FitnessFunctions

%reload_ext autoreload
%autoreload 2

In [2]:
job_graph = JobGraph(file_paths, **job_graph_par)

Cache found loading...done


In [21]:
offer = pd.read_csv("../outputs/0_job_offers.csv").set_index("qId")

In [22]:
es_comp_columns = ["Competence_essential"+str(i) for i in range(0, 3 + 1)]
op_comp_columns = ["Competence_optional"+str(i) for i in range(0, 2 + 1)]
es_know_columns = ["Knowledge_essential"+str(i) for i in range(0, 3 + 1)]
op_know_columns = ["Knowledge_optional"+str(i) for i in range(0, 2 + 1)]

In [23]:
metadata = pd.json_normalize(offer["metadata"].apply(ast.literal_eval))

In [24]:
offer_md = pd.concat([offer, metadata], axis=1)
offer_md.index.rename("qId", inplace=True)
offer_md = offer_md[["uri_competence_essential","uri_competence_optional","uri_knowledge_essential","uri_knowledge_optional"]]

In [25]:
qId_error_comp_ess = []
qId_error_comp_opt = []
qId_error_know_ess = []
qId_error_know_opt = []

In [52]:
counter = np.zeros((4,2))

for qId, job in tqdm(offer.iterrows(),total=len(offer)):
    
    essential_comp = FitnessFunctions.filter(job[es_comp_columns].to_list())
    optional_comp = FitnessFunctions.filter(job[op_comp_columns].to_list())
    essential_know = FitnessFunctions.filter(job[es_know_columns].to_list())
    optional_know = FitnessFunctions.filter(job[op_know_columns].to_list())

    un_es_comp, amb_es_comp = job_graph.skill_test(essential_comp)
    un_op_comp, amb_op_comp = job_graph.skill_test(optional_comp)
    un_es_know, amb_es_know = job_graph.skill_test(essential_know)
    un_op_know, amb_op_know = job_graph.skill_test(optional_know)
    
    contex = un_es_comp + un_op_comp + un_es_know + un_op_know
    
    amb_es_comp = job_graph.de_ambigue(amb_es_comp, contex,True)
    real_uri = set(offer_md.loc[qId, "uri_competence_essential"])
    equal_ = 0 if un_es_comp+ amb_es_comp  ==  real_uri else 1
    counter[0, equal_] += 1
    if equal_ == 1:
        qId_error_comp_ess.append(qId)


    amb_op_comp = job_graph.de_ambigue(amb_op_comp, contex,True)
    real_uri = set(offer_md.loc[qId, "uri_competence_optional"])
    equal_ = 0 if un_op_comp + amb_op_comp  ==  real_uri else 1
    counter[1, equal_] += 1
    if equal_ == 1:
        qId_error_comp_ess.append(qId)
     
    amb_es_know = job_graph.de_ambigue(amb_es_know, contex,True)
    real_uri = set(offer_md.loc[qId, "uri_knowledge_essential"])
    equal_ = 0 if un_es_know + amb_es_know  ==  real_uri else 1
    counter[2, equal_] += 1
    if equal_ == 1:
        qId_error_comp_ess.append(qId)


    amb_op_know = job_graph.de_ambigue(amb_op_know, contex,True)
    real_uri = set(offer_md.loc[qId, "uri_knowledge_optional"])
    equal_ = 0 if un_op_know + amb_op_know  ==  real_uri else 1
    counter[3, equal_] += 1
    if equal_ == 1:
        qId_error_comp_ess.append(qId)
    


100%|██████████| 220/220 [00:00<00:00, 382.54it/s]


In [53]:
counter

array([[  0., 220.],
       [  0., 220.],
       [  0., 220.],
       [  0., 220.]])

In [28]:
qId_error_comp_ess

[173]

In [35]:
essential_comp = FitnessFunctions.filter(offer.loc[173,es_comp_columns].to_list())
print("Essential competence", essential_comp)

std_ess_comp = job_graph.skill_standardize(essential_comp, to_ids=True)
print("Standardized essential competence", std_ess_comp)

real_uri = set(offer_md.loc[173, "uri_competence_essential"])
print("Real essential competence",real_uri)

Essential competence ['supervise staff', 'estimate duration of work', 'ensure cross-department cooperation']
Standardized essential competence {'6d25ef0c-9f6c-4e46-acf3-b2ae022f85f6', 'e207163b-7963-4c3e-9494-7a4bb000211b', 'b216d9a2-7d4e-4312-8d05-e375f4ab44d8'}
Real essential competence {'e207163b-7963-4c3e-9494-7a4bb000211b', 'b216d9a2-7d4e-4312-8d05-e375f4ab44d8', '9601b2a8-68e4-4906-9306-a23679b8f0b0'}


In [42]:
contex = ['e207163b-7963-4c3e-9494-7a4bb000211b', 'b216d9a2-7d4e-4312-8d05-e375f4ab44d8']

In [41]:
job_graph.sys_label2uri["supervise staff"]

['6d25ef0c-9f6c-4e46-acf3-b2ae022f85f6',
 '9581639c-e521-4d12-9b4e-25faca24d34b',
 '9601b2a8-68e4-4906-9306-a23679b8f0b0']

In [44]:
job_graph.node_similarity(job_graph.sys_label2uri["supervise staff"], contex,ids=True)

[0.03571428571428571, 0.0, 0.020130081300813008]